In [1]:
import pandas as pd
from binance.client import Client
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_datareader.data as web
import pandas_ta as ta

In [3]:
from scipy.signal import argrelextrema
from collections import deque

def getHigherLows(data: np.array, order=5, K=2):
    '''
  Finds consecutive higher lows in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive lows need to be higher.
  '''
  # Get lows
    low_idx = argrelextrema(data, np.less, order=order)[0]
    lows = data[low_idx]
  # Ensure consecutive lows are higher than previous lows
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(low_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if lows[i] < lows[i-1]:
            ex_deque.clear()
        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())
    return extrema
def getLowerHighs(data: np.array, order=5, K=2):
    '''
  Finds consecutive lower highs in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive highs need to be lower.
  '''
  # Get highs
    high_idx = argrelextrema(data, np.greater, order=order)[0]
    highs = data[high_idx]
  # Ensure consecutive highs are lower than previous highs
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(high_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if highs[i] > highs[i-1]:
            ex_deque.clear()
        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())
    return extrema
def getHigherHighs(data: np.array, order=5, K=2):
    '''
  Finds consecutive higher highs in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive highs need to be higher.
   '''
  # Get highs
    high_idx = argrelextrema(data, np.greater, order=5)[0]
    highs = data[high_idx]
  # Ensure consecutive highs are higher than previous highs
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(high_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if highs[i] < highs[i-1]:
            ex_deque.clear()
        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())
    return extrema
def getLowerLows(data: np.array, order=5, K=2):
    '''
  Finds consecutive lower lows in price pattern.
  Must not be exceeded within the number of periods indicated by the width 
  parameter for the value to be confirmed.
  K determines how many consecutive lows need to be lower.
  '''
  # Get lows
    low_idx = argrelextrema(data, np.less, order=order)[0]
    lows = data[low_idx]
  # Ensure consecutive lows are lower than previous lows
    extrema = []
    ex_deque = deque(maxlen=K)
    for i, idx in enumerate(low_idx):
        if i == 0:
            ex_deque.append(idx)
            continue
        if lows[i] > lows[i-1]:
            ex_deque.clear()
        ex_deque.append(idx)
        if len(ex_deque) == K:
            extrema.append(ex_deque.copy())
    return extrema

In [4]:
# function to identify signal
def gap(s):
    
    previos_val = s['shifted_hist']
    current_val = s['histogram']
    
    if previos_val is np.nan:
        return 'Null'
    if (previos_val < 0) and (current_val >= 0):
        return 'Shift'
    elif (previos_val > 0) and (current_val <= 0):
        return 'Shift'
    else:
        return 'Null'

In [5]:
df = pd.read_csv('BTCUSDT_3years.csv')
df.index = [dt.datetime.fromtimestamp(x/1000.0) - dt.timedelta(minutes=30, hours=5)
              for x in df.open_time]
df.drop(['open_time', 'open', 'high', 'low', 'volume', 'close_time',
       'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol', 'ignore'],
        inplace=True, axis=1)
df.columns = ['Close']
#print(df)

#MTF EMA
df['EMA_15'] = df['Close'].iloc[2::3].ewm(span=50, adjust=False, min_periods=50).mean()
df['EMA_1h'] = df['Close'].iloc[17::12].ewm(span=50, adjust=False, min_periods=50).mean()

df['EMA_15'].fillna(method='ffill', inplace=True)
df['EMA_1h'].fillna(method='ffill', inplace=True)
#print(df)

#MACD Indicator
k = df['Close'].ewm(span=12, adjust=False, min_periods=12).mean()
d = df['Close'].ewm(span=26, adjust=False, min_periods=26).mean()
df['MACD'] = k - d
df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False, min_periods=9).mean()
df['histogram'] = df['MACD'] - df['MACD_signal']
#print(df)

#for recording Gap in histogram
df['shifted_hist'] = df['histogram'].shift(1)
df['histogram_2g'] = df[['histogram', 'shifted_hist']].apply(gap, axis=1)
df['histogram_2g_bi'] = np.where(df['histogram_2g']=='Shift', 1, 0)
df['histogram_2g_cumsum'] = df['histogram_2g_bi'].cumsum()
#print(df)

#Conditions for Bullish Divergence
df['EMA_15>EMA_1h'] = df['EMA_15'] > df['EMA_1h']
df['MACD<0'] = df['MACD']<0
df['MACD_signal<0'] = df['MACD_signal']<0
#df

#Conditions for Bearish Divergence
df['EMA_15<EMA_1h'] = df['EMA_15'] < df['EMA_1h']
df['MACD>0'] = df['MACD']>0
df['MACD_signal>0'] = df['MACD_signal']>0
#df

#get track of integer index 
df['indx'] = pd.RangeIndex(len(df))


In [29]:
#df.to_csv('models/df_1.csv')

In [264]:
#Bullish Divergence
order = 1
K = 2

close = df['Close'].values
macd = df['MACD'].values

#Bullish Divergence
ll_close = getLowerLows(close, order, K)
hl_macd = getHigherLows(macd, order, K)

#Bearish Divergence
hh_close = getHigherHighs(close, order, K)
lh_macd = getLowerHighs(macd, order, K)

divergences_bullish = []
divergences_bearish = []

#Bullish
for pair1 in ll_close:
    for pair2 in hl_macd:
        s1 = pair2[-1]-pair1[-1]<2
        s2 = pair2[-1]-pair1[-1]>-1
        #s3 = pair2[0]-pair1[0]<5
        #s4 = pair2[0]-pair1[0]>-5
        s5 = pair1[-1] > pair2[0]
        if s1 and s2 and s5:# and s3 and s4:
            divergences_bullish.append((pair1, pair2))
            
#Bearish 
for pair1 in hh_close:
    for pair2 in lh_macd:
        s1 = pair2[-1]-pair1[-1]<2
        s2 = pair2[-1]-pair1[-1]>-1
        #s3 = pair2[0]-pair1[0]<5
        #s4 = pair2[0]-pair1[0]>-5
        s5 = pair1[-1] > pair2[0]
        if s1 and s2 and s5:# and s3 and s4:
            divergences_bearish.append((pair1, pair2))
            
#divergences_bearish

In [265]:
df_bullish_pair = pd.DataFrame()
df_bullish_pair['Close_range'] = [p[0] for p in divergences_bullish]
df_bullish_pair['MACD_range'] = [p[-1] for p in divergences_bullish]
df_bullish_pair.to_csv('models/bup_7')

df_bearish_pair = pd.DataFrame()
df_bearish_pair['Close_range'] = [p[0] for p in divergences_bearish]
df_bearish_pair['MACD_range'] = [p[-1] for p in divergences_bearish]
df_bearish_pair.to_csv('models/bep_7')

In [266]:
df_bullish_pair, df_bearish_pair

(           Close_range        MACD_range
 0             [30, 36]          [31, 37]
 1             [69, 77]          [67, 78]
 2             [91, 97]          [78, 97]
 3           [181, 184]        [175, 184]
 4           [267, 269]        [267, 269]
 ...                ...               ...
 7846  [314392, 314398]  [314388, 314398]
 7847  [314426, 314431]  [314422, 314431]
 7848  [314437, 314442]  [314431, 314442]
 7849  [314449, 314451]  [314442, 314451]
 7850  [314521, 314526]  [314519, 314526]
 
 [7851 rows x 2 columns],
            Close_range        MACD_range
 0           [151, 161]        [143, 162]
 1           [305, 312]        [305, 312]
 2           [491, 520]        [515, 520]
 3           [823, 836]        [817, 837]
 4           [919, 930]        [919, 930]
 ...                ...               ...
 1126  [313279, 313304]  [313300, 313304]
 1127  [313583, 313593]  [313583, 313593]
 1128  [313632, 313671]  [313667, 313671]
 1129  [313793, 313799]  [313794, 313800]
 1130 

In [267]:
cols = ['Type', 'Close_range', 'MACD_range', 'Shift_count',
        'Con_1', 'Con_2', 'Con_3', 'entry', 'entry_close', 'PandL']

In [268]:
def shift_count(macd_range):
    s = macd_range[0]
    e = macd_range[-1]
    dif = df['histogram_2g_cumsum'][e]-df['histogram_2g_cumsum'][s]
    return dif

df_bullish_pair['Shift_count'] = df_bullish_pair['MACD_range'].apply(shift_count)
df_bearish_pair['Shift_count'] = df_bearish_pair['MACD_range'].apply(shift_count)
#df_points

In [269]:
df_bullish_pair['Shift_count'].value_counts(), df_bearish_pair['Shift_count'].value_counts() 

(1    3820
 0    2187
 2    1844
 Name: Shift_count, dtype: int64,
 2    470
 1    348
 0    313
 Name: Shift_count, dtype: int64)

In [270]:
#len(df_points[df_points['Type'] == 'Bull'])
def diff_2nd_point(s):
    e1 = s['Close_range'][-1]
    e2 = s['MACD_range'][-1]
    return e2 - e1

def diff_1st_point(s):
    e1 = s['Close_range'][0]
    e2 = s['MACD_range'][0]
    return e2 - e1

print(df_bullish_pair[['Close_range', 'MACD_range']].apply(diff_2nd_point, axis=1).value_counts())

print(df_bullish_pair[['Close_range', 'MACD_range']].apply(diff_1st_point, axis=1).value_counts())

0    5638
1    2213
dtype: int64
 0     944
-3     787
-4     725
-2     718
-5     648
-6     585
-7     559
-8     468
-9     350
-10    326
-11    274
-12    192
-13    182
-14    168
 1     158
-15    119
-17     93
-16     84
-18     67
-19     50
-20     48
-21     34
-24     31
-23     30
-25     29
-22     27
 2      26
-1      17
-27     16
-28     16
-26     13
-29     13
-30     10
-33      8
-32      7
-36      4
 3       4
-41      4
-31      3
-40      2
-35      2
-38      2
-39      1
-37      1
 10      1
 4       1
-34      1
-42      1
 5       1
-51      1
dtype: int64


In [271]:
_ = df_bullish_pair[['Close_range', 'MACD_range']].apply(diff_1st_point, axis=1).value_counts()
_

 0     944
-3     787
-4     725
-2     718
-5     648
-6     585
-7     559
-8     468
-9     350
-10    326
-11    274
-12    192
-13    182
-14    168
 1     158
-15    119
-17     93
-16     84
-18     67
-19     50
-20     48
-21     34
-24     31
-23     30
-25     29
-22     27
 2      26
-1      17
-27     16
-28     16
-26     13
-29     13
-30     10
-33      8
-32      7
-36      4
 3       4
-41      4
-31      3
-40      2
-35      2
-38      2
-39      1
-37      1
 10      1
 4       1
-34      1
-42      1
 5       1
-51      1
dtype: int64

In [272]:
_.sum()

7851

In [273]:
def condition(macd_range, Type, x):
    s = macd_range[0]
    e = macd_range[-1]
    
    df_hh = df.iloc[s:e+1]
    
    if Type == 'Bull':
        con1 = 'EMA_15>EMA_1h'
        con2 = 'MACD<0'
        con3 = 'MACD_signal<0'
    if Type == 'Bear':
        con1 = 'EMA_15<EMA_1h'
        con2 = 'MACD>0'
        con3 = 'MACD_signal>0'
        
    if x == 1:    
        condition_1 = df_hh[con1].sum() == len(df_hh)
        return condition_1
    if x == 2:
        condition_2 = df_hh[con2].sum() == len(df_hh)
        return condition_2
    if x == 3:
        condition_3 = df_hh[con3].sum() == len(df_hh)
        return condition_3

In [274]:
#Condition 1 on MTF EMA
df_bullish_pair['Con_1'] = df_bullish_pair['MACD_range'].apply(condition, Type='Bull', x=1)
df_bearish_pair['Con_1'] = df_bearish_pair['MACD_range'].apply(condition, Type='Bear', x=1)
#df_points

#Condition 2
df_bullish_pair['Con_2'] = df_bullish_pair['MACD_range'].apply(condition, Type='Bull', x=2)
df_bearish_pair['Con_2'] = df_bearish_pair['MACD_range'].apply(condition, Type='Bear', x=2)
df_points

df_bullish_pair['Con_3'] = df_bullish_pair['MACD_range'].apply(condition, Type='Bull', x=3)
df_bearish_pair['Con_3'] = df_bearish_pair['MACD_range'].apply(condition, Type='Bear', x=3)
df_points

,Type,Close_range,MACD_range,Shift_count,Con_1,Con_2,Con_3,Conditions
0,Bull,"[30, 36]","[31, 37]",1,False,False,False,False
1,Bull,"[39, 44]","[45, 47]",0,False,False,False,False
2,Bull,"[69, 77]","[67, 78]",2,False,True,True,False
3,Bull,"[91, 97]","[78, 97]",1,False,False,False,False
4,Bull,"[181, 184]","[175, 184]",1,False,True,True,False
...,...,...,...,...,...,...,...,...
20039,Bear,"[314004, 314017]","[314018, 314022]",0,False,False,False,False
20040,Bear,"[314378, 314389]","[314376, 314394]",2,True,True,True,True
20041,Bear,"[314429, 314481]","[314481, 314487]",2,False,True,True,False
20042,Bear,"[314514, 314524]","[314506, 314524]",2,False,False,True,False


In [275]:
df_bullish_pair['Conditions'] = (df_bullish_pair['Con_1'] & 
                                 df_bullish_pair['Con_2'] & 
                                 df_bullish_pair['Con_3'])

df_bearish_pair['Conditions'] = (df_bearish_pair['Con_1'] & 
                                 df_bearish_pair['Con_2'] & 
                                 df_bearish_pair['Con_3'])
df_bullish_pair, df_bearish_pair

(           Close_range        MACD_range  Shift_count  Con_1  Con_2  Con_3  \
 0             [30, 36]          [31, 37]            1  False  False  False   
 1             [69, 77]          [67, 78]            2  False   True   True   
 2             [91, 97]          [78, 97]            1  False  False  False   
 3           [181, 184]        [175, 184]            1  False   True   True   
 4           [267, 269]        [267, 269]            0  False  False  False   
 ...                ...               ...          ...    ...    ...    ...   
 7846  [314392, 314398]  [314388, 314398]            2  False  False  False   
 7847  [314426, 314431]  [314422, 314431]            1  False   True   True   
 7848  [314437, 314442]  [314431, 314442]            0  False  False  False   
 7849  [314449, 314451]  [314442, 314451]            0  False  False  False   
 7850  [314521, 314526]  [314519, 314526]            1   True  False  False   
 
       Conditions  
 0          False  
 1        

In [276]:
cols = ['Close_range', 'MACD_range']

mask11 = df_bullish_pair['Shift_count'] >= 2
mask12 = df_bullish_pair['Conditions']
df_bullish_final = df_bullish_pair.loc[mask11 & mask12, cols].copy()

mask21 = df_bearish_pair['Shift_count'] >= 2
mask22 = df_bearish_pair['Conditions']
df_bearish_final = df_bearish_pair.loc[mask21 & mask22, cols].copy()

df_bullish_final, df_bearish_final

(           Close_range        MACD_range
 39        [1994, 2002]      [1994, 2002]
 49        [2339, 2341]      [2322, 2341]
 50        [2344, 2348]      [2341, 2348]
 58        [2655, 2658]      [2643, 2658]
 59        [2661, 2663]      [2658, 2664]
 ...                ...               ...
 7645  [305957, 305961]  [305952, 305961]
 7698  [308234, 308237]  [308224, 308238]
 7777  [311379, 311386]  [311379, 311386]
 7812  [312626, 312632]  [312612, 312633]
 7836  [313813, 313821]  [313813, 313822]
 
 [455 rows x 2 columns],
            Close_range        MACD_range
 5         [1562, 1574]      [1566, 1574]
 20        [6667, 6679]      [6666, 6680]
 21        [7255, 7270]      [7255, 7271]
 24        [8821, 8836]      [8822, 8837]
 25        [8846, 8854]      [8846, 8854]
 ...                ...               ...
 1102  [307240, 307249]  [307241, 307250]
 1104  [307708, 307721]  [307715, 307721]
 1117  [310962, 310975]  [310967, 310976]
 1118  [310985, 310997]  [310976, 310998]
 1126  

In [277]:
def entry(macd_range):
    e = macd_range[-1]
    mask = df['histogram_2g_cumsum'] == df['histogram_2g_cumsum'][e] + 1
    entry_point = df.loc[mask, 'indx'][0]
    return entry_point

df_bullish_final['entry'] = df_bullish_final['MACD_range'].apply(entry)
df_bearish_final['entry'] = df_bearish_final['MACD_range'].apply(entry)

df_bullish_final, df_bearish_final

(           Close_range        MACD_range   entry
 39        [1994, 2002]      [1994, 2002]    2004
 49        [2339, 2341]      [2322, 2341]    2345
 50        [2344, 2348]      [2341, 2348]    2349
 58        [2655, 2658]      [2643, 2658]    2659
 59        [2661, 2663]      [2658, 2664]    2665
 ...                ...               ...     ...
 7645  [305957, 305961]  [305952, 305961]  305963
 7698  [308234, 308237]  [308224, 308238]  308241
 7777  [311379, 311386]  [311379, 311386]  311389
 7812  [312626, 312632]  [312612, 312633]  312636
 7836  [313813, 313821]  [313813, 313822]  313823
 
 [455 rows x 3 columns],
            Close_range        MACD_range   entry
 5         [1562, 1574]      [1566, 1574]    1576
 20        [6667, 6679]      [6666, 6680]    6682
 21        [7255, 7270]      [7255, 7271]    7273
 24        [8821, 8836]      [8822, 8837]    8844
 25        [8846, 8854]      [8846, 8854]    8855
 ...                ...               ...     ...
 1102  [307240, 307249]

In [278]:
(df_bullish_final['Close_range'].value_counts().value_counts(),
 df_bearish_final['Close_range'].value_counts().value_counts())

(1    455
 Name: Close_range, dtype: int64,
 1    180
 Name: Close_range, dtype: int64)

In [279]:
(df_bullish_final['MACD_range'].value_counts().value_counts(),
 df_bearish_final['MACD_range'].value_counts().value_counts())

(1    455
 Name: MACD_range, dtype: int64,
 1    180
 Name: MACD_range, dtype: int64)

In [280]:
(df_bullish_final['entry'].value_counts().value_counts(),
 df_bearish_final['entry'].value_counts().value_counts())

(1    455
 Name: entry, dtype: int64,
 1    180
 Name: entry, dtype: int64)

In [281]:
# function to identify signal
def repeated_entry(s):
    
    previous_val = s['entry_shifted']
    current_val = s['entry']
    
    if previous_val == current_val:
        return 1
    else:
        return 0
    

In [282]:
df_bullish_final['entry_shifted'] = df_bullish_final['entry'].shift(1)
df_bullish_final['repeated_entry'] = df_bullish_final[['entry', 'entry_shifted']].apply(repeated_entry, axis=1)
#indx_to_remove = df_bullish_final['repeated_entry'] == 1
#df_final.drop(indx_to_remove, axis=0, inplace=True)

df_bearish_final['entry_shifted'] = df_bearish_final['entry'].shift(1)
df_bearish_final['repeated_entry'] = df_bearish_final[['entry', 'entry_shifted']].apply(repeated_entry, axis=1)
#indx_to_remove = df_bearish_final['repeated_entry'] == 1

In [283]:
df_bullish_final['entry_close'] = df_bullish_final['entry'].apply(lambda i: df.iloc[i]['Close'])
df_bearish_final['entry_close'] = df_bearish_final['entry'].apply(lambda i: df.iloc[i]['Close'])


In [284]:
def exit(entry_point, Type):
    close = df.iloc[entry_point]['Close']
    exit_point = entry_point + 1
    exit = df.iloc[exit_point]['Close']
    diff = exit - close
    stop_loss = 1.5*close/100
    target = 1.5*stop_loss
    #print(entry_point)
    if Type == 'Bull':
        while diff <= target and diff>=-stop_loss:
            exit_point += 1
            #print('\t', exit_point)
            exit = df.iloc[exit_point]['Close']
            diff = exit - close
        if not diff<=target:
            return (exit_point, exit, 'profit')
        if not diff>=-stop_loss:
            return (exit_point, exit, 'loss')
    if Type == 'Bear':
        while diff >= -target and diff <= stop_loss:
            exit_point += 1
            exit = df.iloc[exit_point]['Close']
            diff = exit - close
        if not diff >= -target:
            return (exit_point, exit, 'profit')
        if not diff <= stop_loss:
            return (exit_point, exit, 'loss')
        

In [285]:
df_bullish_final['exit'] = df_bullish_final['entry'].apply(exit, Type='Bull')
df_bearish_final['exit'] = df_bearish_final['entry'].apply(exit, Type='Bear')

df_bullish_final, df_bearish_final

(           Close_range        MACD_range   entry  entry_shifted  \
 39        [1994, 2002]      [1994, 2002]    2004            NaN   
 49        [2339, 2341]      [2322, 2341]    2345         2004.0   
 50        [2344, 2348]      [2341, 2348]    2349         2345.0   
 58        [2655, 2658]      [2643, 2658]    2659         2349.0   
 59        [2661, 2663]      [2658, 2664]    2665         2659.0   
 ...                ...               ...     ...            ...   
 7645  [305957, 305961]  [305952, 305961]  305963       305519.0   
 7698  [308234, 308237]  [308224, 308238]  308241       305963.0   
 7777  [311379, 311386]  [311379, 311386]  311389       308241.0   
 7812  [312626, 312632]  [312612, 312633]  312636       311389.0   
 7836  [313813, 313821]  [313813, 313822]  313823       312636.0   
 
       repeated_entry  entry_close                        exit  
 39                 0      3634.99       (3930, 3577.67, loss)  
 49                 0      3655.49       (2883, 3590

In [286]:
df_bearish_final['repeated_entry'].value_counts()

0    180
Name: repeated_entry, dtype: int64

In [287]:
df_bullish_final['repeated_entry'].value_counts()

0    455
Name: repeated_entry, dtype: int64

In [288]:
df_bullish_final['exit_point'] = df_bullish_final['exit'].apply(lambda t: t[0])
df_bullish_final['exit_close'] = df_bullish_final['exit'].apply(lambda t: t[1])
df_bullish_final['PandL'] = df_bullish_final['exit'].apply(lambda t: t[-1])

df_bearish_final['exit_point'] = df_bearish_final['exit'].apply(lambda t: t[0])
df_bearish_final['exit_close'] = df_bearish_final['exit'].apply(lambda t: t[1])
df_bearish_final['PandL'] = df_bearish_final['exit'].apply(lambda t: t[-1])

df_bullish_final, df_bearish_final

(           Close_range        MACD_range   entry  entry_shifted  \
 39        [1994, 2002]      [1994, 2002]    2004            NaN   
 49        [2339, 2341]      [2322, 2341]    2345         2004.0   
 50        [2344, 2348]      [2341, 2348]    2349         2345.0   
 58        [2655, 2658]      [2643, 2658]    2659         2349.0   
 59        [2661, 2663]      [2658, 2664]    2665         2659.0   
 ...                ...               ...     ...            ...   
 7645  [305957, 305961]  [305952, 305961]  305963       305519.0   
 7698  [308234, 308237]  [308224, 308238]  308241       305963.0   
 7777  [311379, 311386]  [311379, 311386]  311389       308241.0   
 7812  [312626, 312632]  [312612, 312633]  312636       311389.0   
 7836  [313813, 313821]  [313813, 313822]  313823       312636.0   
 
       repeated_entry  entry_close                        exit  exit_point  \
 39                 0      3634.99       (3930, 3577.67, loss)        3930   
 49                 0     

In [289]:
(df_bullish_final['exit_point'].value_counts().value_counts(), 
 df_bearish_final['exit_point'].value_counts().value_counts())

(1    329
 2     42
 3     10
 4      3
 Name: exit_point, dtype: int64,
 1    157
 2      7
 3      3
 Name: exit_point, dtype: int64)

In [290]:
df_bullish_final['PandL'].value_counts(), df_bearish_final['PandL'].value_counts()

(profit    229
 loss      226
 Name: PandL, dtype: int64,
 loss      110
 profit     70
 Name: PandL, dtype: int64)

In [291]:
vc = df_bullish_final['PandL'].value_counts() + df_bearish_final['PandL'].value_counts()
vc

loss      336
profit    299
Name: PandL, dtype: int64

In [292]:
def PLsum(s):
    if s == 'profit':
        return 1 + 1.5*0.015
    if s == 'loss':
        return 1 - 0.015
    

In [293]:
s = (df_bullish_final['PandL'].map(PLsum).cumprod().iloc[-1]*
     df_bearish_final['PandL'].map(PLsum).cumprod().iloc[-1])
s

4.829532959989992

In [294]:
l = len(df_bullish_final) + len(df_bearish_final)

In [295]:
pp, pf = vc['profit']/vc.sum(), (1.5*1.5*vc['profit'])/(1.5*vc['loss'])

In [296]:
l, pp, s-1, pf

(635, 0.47086614173228347, 3.829532959989992, 1.3348214285714286)